In [ ]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import os
import re
import sys
from datetime import datetime
import memory_profiler
import gc
from multiprocessing import Process
import multiprocessing as mp

sys.path.insert(0, '../../drlib')
import drlib as dr

%matplotlib widget

In [6]:
dataDir = '/drBiggerBoy/drData/Data/'
fileList = os.listdir(dataDir)
fileList.sort(key=lambda f: int(re.sub('\D', '', f)))

fileName = 'preProcDataSet.hdf5'

In [109]:
#pack "searchable database" df
#indexed by int which counts spectra
#Columns: 
    #File, measData, Date, Temperture, West, Vertical, South, Phi, Theta, VetoSpec(array, len=1e4)

database = pd.read_csv('databaseForPandas.txt', skiprows=0)
database['Date'] = pd.to_datetime(database['Date'], infer_datetime_format=True)
database.set_index('Date', inplace=True)
#database['Rigol'] = 

#print(database.tail())

'''
#pack all veto data into DF
dataDir = '/drBiggerBoy/drData/Data/'
fileList = os.listdir(dataDir)
fileList.sort(key=lambda f: int(re.sub('\D', '', f)))

rigolTempArr = np.zeros((10000, 4))
rigolIdx = 0

for file in fileList[0:5]:
    print(file)
    dataset = h5py.File(dataDir +  file, 'r')
    for measData in dataset:



        datetimeDf     = pd.read_hdf(dataDir + file, key = measData, start=0, stop=1)
        dateTimeStr = datetimeDf.columns[0][0]
        dateTime    = datetime.strptime(dateTimeStr, '%Y-%m-%d %H:%M:%S.%f')


        for key in measDataSubKeys:
            #print(key)
            if key[0] == 'R':
                #print('keys = \n', dataset[measData])
                #rigolTempArr[:,rigolIdx] = dataset[measData][key]
                #print(rigolTempArr)
                rigolIdx += 1
                #print(database[dateTime]) #= np.array(h5py.File(dataDir +  file, 'r')[measData][key])
'''






#check that dates are consecutive
dates = database.index
date_ints = set([d.toordinal() for d in dates])
if max(date_ints) - min(date_ints) == len(date_ints) - 1:
    print("Dates are consecutive")
else:
    raise Exception('Dates are not consecutive')



Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are consecutive
Dates are 

In [49]:
database[datetime.strptime('2022-07-07 12:46:11.184000')]

TypeError: strptime() takes exactly 2 arguments (1 given)

In [44]:
from numpy import s_
sliceObject = s_['2022-7-7 12:30':'2022-7-7 13:04':None, 
                '2022-7-7 14:00':'2022-7-7 17:20':None]

measDataList = []
for slice in sliceObject:
    print(slice)
    #print(database[slice])
    measDataListTemp = (database[slice]['measData'].to_list())
    measDataList.extend(measDataListTemp)
#database[sliceObject]['measData']
print(measDataList)


#f.close()
newDataDir = '/drBiggerBoy/drData/run1Data/'
f = h5py.File(newDataDir + fileName, 'r')

a = np.zeros(8000000)
for measData in measDataList:
    a+=(f[str(measData)]['diffSpecWatts'][1000:8001000])
#print(a)

plt.close('all')
#plt.plot(a)

slice('2022-7-7 12:30', '2022-7-7 13:04', None)
slice('2022-7-7 14:00', '2022-7-7 17:20', None)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 2

In [ ]:
database['2022-7-8 00':'2022-7-8 00:04']['measData'].to_list()

In [ ]:
#time since start plot
database.iloc[0]

In [ ]:
f = h5py.File('/drBiggerBoy/drData/Data/data_0.h5', 'r')
type(f['measdata_0']['table'][1][1][0])

In [ ]:
f = h5py.File('/drBiggerBoy/drData/run1DataTest/' + fileName, 'r')
keys = f.keys()

for spec in list(keys)[0:1]:
    a = f[spec]['diffSpecWatts'][1]
    print(type(a))
f.close()

In [164]:
#pack new h5 file 
f.close()
newDataDir = '/drBiggerBoy/drData/run1Data/'
f = h5py.File(newDataDir + fileName, 'w')
numFiles        = 0
numMeasData     = 0
numRigolSpec    = 0


########################################
#Main loop
########################################
for file in fileList:
    print(file)
    dataset         = h5py.File(dataDir +  file, 'r')
    numFiles        += 1
    measDataKeys    = list(dataset.keys())
    measDataKeys.sort(key=lambda f: int(re.sub('\D', '', f)))

    for measData in measDataKeys:
        measDataSubKeys = dataset[measData].keys()
        #print(measDataSubKeys)
        datasetDf   = pd.read_hdf(dataDir + file, key = measData)
        dateTimeStr = datasetDf.columns[0][0]
        dateTime    = datetime.strptime(dateTimeStr, '%Y-%m-%d %H:%M:%S.%f')
        diffSpec    = np.float32(dr.fft2Watts(np.asarray(datasetDf.iloc[:,2][1:])))

        grp         = f.create_group(str(numMeasData))
        grp.attrs.create('Datetime', dateTimeStr)
        for attr in database.keys():
            grp.attrs.create(attr, database.loc[dateTime][attr])
        
        dset            = grp.create_dataset('diffSpec_W', data = diffSpec)

        numMeasData     += 1
        rigolIdx        = 0
        rigolTempArr    = np.zeros((10000,4))
        for key in measDataSubKeys:
            if key[0]=='R':
                rigolTempArr[:, rigolIdx] = (np.float32(dataset[measData][key]))
                rigolIdx += 1
        rigolMaxArr = np.zeros(10000)
        for i, row in enumerate(rigolTempArr):
            rigolMaxArr[i] = -1*((-1*row).max())
            
        rigolDest    = grp.create_dataset('rigolSpec_dBm', data = rigolMaxArr)
            
print('number of files = ', numFiles)
print('number of meas data = ', numMeasData)

f.close()

data_0.h5


In [148]:
arr = np.random.random((100000,4))

maxArr = np.zeros(100000)
for i, row in enumerate(arr):
    maxArr[i] = row.max()


print(maxArr)
#print(arr.transpose.map(&:max))

[0.49893401 0.62124563 0.9666374  ... 0.88878425 0.62671569 0.76373053]


In [149]:
%%timeit
for i, row in enumerate(arr):
    maxArr[i] = row.max()


153 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
f.close()
newDataDir = '/drBiggerBoy/drData/run1Data/'
f = h5py.File(newDataDir + fileName, 'r')

keys=database['2000-1-1':'2022-7-8'].index
print((keys))
keysStr = [str(key)[:-3] for key in keys]
print(keysStr[799])


i=0
#print(f.keys())
for key in keysStr:
    print(f[key]['diffSpecWatts'])
    print(i)
    i+=1


f.close()

In [160]:
f.close()